In [1]:
#########################################
#          TEXT PREDICTION              #
#     Part One: Creating the Model      #
#         Julia Brzustowski             #
#########################################

####################
# import libraries #
####################

# tensorflow for keras and neural network
import tensorflow as tf
print ("TensorFlow version:", tf.__version__)

# keras neural network libraries
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense

# libraries for display widgets
from google.colab import widgets
import ipywidgets
from ipywidgets import Button
from IPython.display import display, clear_output

# numpy for dealing with arrays
import numpy as np

import matplotlib.pyplot as plt


####################
#   Prepare Data   #
####################

# open lyric file with data
file = open("all-songs.txt", "r", encoding = "utf8")
lines=[]


# CREATE DICTIONARY and prepare dataset
# add words to variable
for i in file:
  lines.append(i)

data = ' '.join(lines)

# remove any unnecessary characters - leaving in '\n', newline character will be useful to consider as its own word
data = data.replace('"','').replace(',','')
data = data.lower()

# remove any repeats - need to create index so only one of each word is needed
z = []
for w in data.split():
  if w not in z:
    z.append(w)

dictionary = ' '.join(z)

# use keras tokenizer to create a dictionary of all words
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([dictionary])

# convert the lyrics to numbers based on dictionary index
sequence_data = tokenizer.texts_to_sequences([data])[0]

# store sequences of words (previous three words + word) in a list
sequences = []

for i in range (1, len(sequence_data)):
  words = sequence_data [i-3:i+1]
  sequences.append (words)

#scale inputs so that they are in between 0 and 1
scaler=MinMaxScaler()

scaler.fit(sequences[2:len(sequences)])
sequences=scaler.transform(sequences[2:len(sequences)])


# split data figure out what data is training data and what is testing
train, test = train_test_split(sequences,test_size=0.2,random_state=1)

# create 3 input variables and 1 and output variable
xtrain = []
ytrain = []

xtest = []
ytest = []

count=0
# loop through each sequence
for i in train[2:len(train)]:
  # store previous 3 word in their own input variables, current word in output
  # this will be used for training
  xtrain.append([])
  xtrain[count].append(i[0])
  xtrain[count].append(i[1])
  xtrain[count].append(i[2])
  ytrain.append(i[3])
  count+=1

count=0
# loop through each sequence
for i in test:
  # store previous 3 word in their own input variables, current word in output
  # this will be used for training
  xtest.append([])
  xtest[count].append(i[0])
  xtest[count].append(i[1])
  xtest[count].append(i[2])
  ytest.append(i[3])
  count+=1



################################################
#                MODEL TIME                    #
################################################

# CREATE FUNCTIONAL API MODEL - NEURAL NETWORK using keras
# create and define an input layer
inputLayer = Input (shape=(3,), name='inputLayer')
# connect the layers
# create hidden layers using dense
layer1 = Dense (1000, activation = "relu", name = 'layer1')(inputLayer)
layer2 = Dense (1000, activation = "relu", name = 'layer2')(layer1)
# create output layer
w1output = Dense(1, activation = "linear", name = 'w1output')(layer2)
# create branched layer for multiple outputs
branchedLayer1 = Dense (1000, activation = "relu", name = 'branchedLayer1')(layer2)
w2output = Dense (1, activation = "linear", name = 'w2output')(branchedLayer1)
branchedLayer2 = Dense (1000, activation = "relu", name = 'branchedLayer2')(layer2)
w3output = Dense (1, activation = "linear", name = 'w3output')(branchedLayer2)
# create and define a model using model class
model = Model(inputs=inputLayer, outputs = [w1output,w2output,w3output])
# display model summary for debugging - model.summary()
print (model.summary())

# import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
# modelCheckpoint used to store best weights
checkpoint = ModelCheckpoint('models.h5', monitor='loss',verbose=1, save_best_only=True,mode='auto' )
# Reduce learning rate used when the accuracy doesn't improve after 3 times through the dataset
reduceLR = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.001, verbose=1)
#visualize loss
TBvisual=TensorBoard(log_dir='logs')

#compile the model
model.compile(optimizer='adam',loss={'w1output':'mse','w2output':'mse','w3output':'mse'})
model.fit(xtrain, ytrain, epochs=10, batch_size=64, callbacks=[checkpoint, reduceLR, TBvisual])

# evaluate the model for accuracy - only in debug/model preparation
# run predictions with test portion of data
predictions = model.predict(xtest)[0]
#predictions = scaler.inverse_transform(predictions)

#ytest=scaler.inverse_transform(ytest)
plt.scatter(ytest,predictions)
plt.xlabel('actual word')
plt.ylabel('predicted word')



TensorFlow version: 2.13.0


FileNotFoundError: ignored

In [ ]:
#########################################
#          TEXT PREDICTION              #
#    Part Two: Integrating the Model    #
#         Julia Brzustowski             #
#########################################

# import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

# load model from other cell
model = keras. models.load_model("models.h5")

# numpy for dealing with arrays
import numpy as np

# math for rounding
import math

# libraries for display widgets
from google.colab import widgets
import ipywidgets
from ipywidgets import Button
from IPython.display import display, clear_output
import time


####################################
#   Prepare DICTIONARY and SCALER  #
####################################

# open lyric file with data
file = open("all-songs.txt", "r", encoding = "utf8")
lines=[]

# CREATE DICTIONARY and prepare dataset
# add words to variable
for i in file:
  lines.append(i)

data = ' '.join(lines)

# remove any unnecessary characters - leaving in '\n', newline character will be useful to consider as its own word
data=data.replace('"','').replace(',','').replace('?','').replace('.','').replace('(','').replace(')','')
data = data.lower()

# remove any repeats - need to create index so only one of each word is needed
z = []
for w in data.split():
  if w not in z:
    z.append(w)

dictionary = ' '.join(z)

# use keras tokenizer to create a dictionary of all words
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([dictionary])

# convert the lyrics to numbers based on dictionary index
sequence_data = tokenizer.texts_to_sequences([data])[0]

# store sequences of words (previous three words + word) in a list
sequences = []

for i in range (1, len(sequence_data)):
  words = sequence_data [i-3:i]
  sequences.append (words)
wordSeq=tokenizer.sequences_to_texts(sequences)

#scale inputs so that they are in between 0 and 1
scaler=MinMaxScaler()

scaler.fit(sequences[2:len(sequences)])
sequences=scaler.transform(sequences[2:len(sequences)])

# create 3 input variables and 1 and output variable
x = []
y = []

count=0
# loop through each sequence
for i in sequences[2:len(sequences)]:
  # store previous 3 word in their own input variables, current word in output
  # this will be used for training
  x.append([])
  x[count].append(i[0])
  x[count].append(i[1])
  x[count].append(i[2])
  #y.append(i[3])
  count+=1

###########################
#        Buttons          #
###########################
def makeLayout (grid):
  # make quit button
  qButton = ipywidgets.widgets.Button(description = "Quit",button_style='danger')
  qButton.on_click(qButton_clicked)

  # make newline button
  nlButton = ipywidgets.widgets.Button(description = "New Line",button_style='info')
  nlButton.on_click(nlButton_clicked)

  # make suggest button
  suggestButton = ipywidgets.widgets.Button(description = "Suggest",button_style='success')
  suggestButton.on_click(suggestButton_clicked)

  # display buttons in grid
  for (row,col) in grid:
    if col==0 and row==0:
      display(iWindow)
    if col==1 and row==0:
      display (suggestButton)
    elif col==2 and row==0:
      display (nlButton)
    elif col==3 and row==0:
      display (qButton)

# function for when buttons are clicked
def sButton_clicked(b):
  # display the word from description
  for (row,col) in grid:
    if col == 0 and row ==1:
      # if there's a space, I don't need to add one
      if iWindow.value[len(iWindow.value)-1] == " ":
        print (iWindow.value,b.description, sep='')
        iWindow.value = iWindow.value + b.description
      else:
        print (iWindow.value,b.description)
        iWindow.value = iWindow.value + ' ' + b.description
  # pretend they clicked suggest again so that it updates live
  suggestButton_clicked(b)

# function for suggest button
def suggestButton_clicked(b):
  # get input from text window
  input=iWindow.value[::-1]
  #print (input)
  threeWords=[]
  currWord=''
  count=0
  for i in range(len(input)): # split input into three words
    if count==3:
      break
    if input[i]==' ' and i!=0:
      threeWords.insert(0,currWord[::-1])
      currWord=''
      count+=1
    elif i==len(input)-1:
      currWord+=input[i]
      threeWords.insert(0,currWord[::-1])
      currWord=''
      count+=1
    else:
      currWord+=input[i]
  if count<3: # if they didn't enter enough words, wait for more
    print ("Please enter more words!",end='')
    time.sleep(3)
    print ("\r",' ')
    return -1
  else:
    # if enough words, call preprocessing function with them
    IN = prepareWords (threeWords)
    if IN[1] == False: # make sure all words went through
      print ("Hmm. I don't recognize that word.",end='')
      time.sleep(3)
      print ("\r",' ')
    else:
      # if words went through, get the predictions
      suggestions = predict(IN[0])
      # reset the suggestion buttons
      makeSuggests(suggestions,grid)

# function for quit button
def qButton_clicked(b):
  writtenLines.append(iWindow.value)
  print ("\n", "Writing complete.", sep = "")
  print ('')
  print ("YOUR PROSE:")
  print ('')
  for i in writtenLines:
    print (i)
  writtenLines.clear()

# function for newline button
def nlButton_clicked(b):
  writtenLines.append(iWindow.value)
  # clear iWindow
  iWindow.value = ''
  # add a new line
  for (row,col) in grid:
    if col == 0 and row ==1:
      print ('')
    # reset suggestions
    if col >= 1 and row ==1:
      if once == True:
        sButtons[col-1].close()

# function to prepare input
def prepareWords (words):
  # prepare each word
  for x in range (len(words)):
    words[x]=words[x].replace('"','').replace(',','').replace(' ','').replace('\n','').replace('?','').replace('.','').replace('(','').replace(')','')
    words[x] = words[x].lower()
  try: # try to convert to number, won't work if there's a typo
    inputSeq = tokenizer.texts_to_sequences([words])[0]
    #reshape to convert to between 0 and 1
    inputSeq=np.reshape(inputSeq,(-1,3))
    inputSeq=scaler.transform(inputSeq)
    return (inputSeq,True)
  except:
    return ('fail',False)

# function that gets predictions from model
def predict (input):
  # call model to predict
  predictions = model.predict(input)
  # reshape and convert back to index number
  predictions = np.reshape(predictions,(-1,3))
  predictions = scaler.inverse_transform(predictions)
  #print (predictions)

  # round number to closest integer
  roundedP=[[]]
  for i in range(len(predictions)):
    for j in range(len(predictions[i])):
      roundedP[0].append(round(predictions[i][j]))
  newWord=tokenizer.sequences_to_texts(roundedP)

  #convert words to seperate list items
  currWord=''
  finalWords=[]
  for i in range (len(newWord[0])):
    if newWord[0][i]==' ':
      finalWords.append(currWord)
      currWord=''
    elif i==len(newWord[0])-1:
      currWord+=newWord[0][i]
      finalWords.append(currWord)
      currWord=''
    else:
      currWord+=newWord[0][i]
  # return the final word predictions
  return (finalWords)

def makeSuggests (words,grid):
  global once
  global sButtons

  #check if buttons are already there and delete them if they are
  for (row,col) in grid:
    if col >= 1 and row ==1:
      if once == True:
        sButtons[col-1].close()
  # make 3 buttons and add to list
  for i in range(0,3):
    # if not new
    if once:
      sButtons[i] = ipywidgets.widgets.Button(description=words[i])
      sButtons[i].on_click(sButton_clicked)
    else:
      sbutton = ipywidgets.widgets.Button(description=words[i])
      sbutton.on_click(sButton_clicked)
      sButtons.append(sbutton)

  # display buttons in grid
  for (row,col) in grid:
    if col >= 1 and row ==1:
      display(sButtons[col-1])
  # change once variable so program knows to delete old buttons
  once = True

# rounding function
def round (num):
  num+=0.5
  num=math.floor(num)
  return num

#############################
#          Main Area        #
#############################
grid=widgets.Grid(2,4)
iWindow = ipywidgets.Textarea (description='Start writing:',continuous_update=True)
once = False
sButtons=[]
makeLayout (grid)
print ('')
writtenLines = []



,,,
,,,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Textarea(value='', description='Start writing:')

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='can', style=ButtonStyle())

<IPython.core.display.Javascript object>